## Imports

In [ ]:
import math

import tsplib95
import networkx
import elkai

import plotly.graph_objects as go

import SimTSP

## Helpers

In [ ]:
def get_histogram(instance_filename, std_rate = 0.1, n_simulations = 10000):
    #
    orig_costs = SimTSP.get_cost_matrix(instance_filename)
    #    
    return SimTSP.simulate(orig_costs, std_rate, n_simulations)        

In [ ]:
def plot_histogram(histogram, instance_filename):
    
    # get static optimal solution    
    instance = tsplib95.load(instance_filename)
    costs = networkx.to_numpy_array(instance.get_graph())
    tour = elkai.DistanceMatrix(costs).solve_tsp()
    static_cost = elkai.utils.path_distance(tour, costs)
    
    # Calculate the mean
    total_frequency = sum(histogram.values())  # Sum of all frequencies
    weighted_sum = sum(key * value for key, value in histogram.items())  # Sum of (key * frequency)

    mean = weighted_sum / total_frequency
    # Calculate the weighted variance
    variance_sum = sum(value * ((key - mean) ** 2) for key, value in histogram.items())
    variance = variance_sum / total_frequency

    # Take the square root to get the standard deviation
    std_dev = math.sqrt(variance)
    
    # Extract keys and values
    categories = list(histogram.keys())
    frequencies = list(histogram.values())

    fig = go.Figure()

    # Add bars for the histogram
    fig.add_trace(go.Bar(
        x=categories, 
        y=frequencies, 
        name='Frequencies',
        marker_color='blue',
        opacity=0.7
    ))

    # Add a line for the mean
    fig.add_trace(go.Scatter(
        x=[mean, mean], 
        y=[0, max(frequencies)], 
        mode='lines', 
        name=f'Mean ({mean:.2f})',
        line=dict(color='red', dash='dash')
    ))

    # Add lines for mean ± std_dev
    fig.add_trace(go.Scatter(
        x=[mean - std_dev, mean - std_dev], 
        y=[0, max(frequencies)], 
        mode='lines', 
        name=f'Mean - Std ({(mean - std_dev):.2f})',
        line=dict(color='green', dash='dot')
    ))    

    fig.add_trace(go.Scatter(
        x=[mean + std_dev, mean + std_dev], 
        y=[0, max(frequencies)], 
        mode='lines', 
        name=f'Mean + Std ({(mean + std_dev):.2f})',
        line=dict(color='green', dash='dot')
    ))
    
    # Add lines for mean ± 2 x std_dev
    fig.add_trace(go.Scatter(
        x=[mean - 2 * std_dev, mean - 2 * std_dev], 
        y=[0, max(frequencies)], 
        mode='lines', 
        name=f'Mean - 2 x Std ({(mean - 2 * std_dev):.2f})',
        line=dict(color='blue', dash='dot')
    ))    

    fig.add_trace(go.Scatter(
        x=[mean + 2 * std_dev, mean + 2 * std_dev], 
        y=[0, max(frequencies)], 
        mode='lines', 
        name=f'Mean + 2 x Std ({(mean + 2 * std_dev):.2f})',
        line=dict(color='blue', dash='dot')
    ))
    
    # Add lines for static cost
    fig.add_trace(go.Scatter(
        x=[static_cost, static_cost], 
        y=[0, max(frequencies)], 
        mode='lines', 
        name=f'Optimal static cost ({(static_cost):.2f})',
        line=dict(color='black', dash='dot')
    )) 

    # Customize layout
    fig.update_layout(
        title="Histogram with Mean and Standard Deviation",
        xaxis_title="Costs",
        yaxis_title="Frequencies",
        template="plotly_white"
    )

    # Show the plot
    fig.show()

In [ ]:
def run(instance_filename):
    histogram = get_histogram(instance_filename)
    plot_histogram(histogram, instance_filename)

## Run

In [ ]:
instance_filename = "data/ch150.tsp"
run(instance_filename)

In [ ]:
instance_filename = "data/berlin52.tsp"
run(instance_filename)

In [ ]:
instance_filename = "data/a280.tsp"
run(instance_filename)